In [1]:
import dgl

ratings = dgl.heterograph(
    {('user', '+1', 'movie') : [(0, 0), (0, 1), (1, 0)],
     ('user', '-1', 'movie') : [(2, 1)]})

In [2]:
import scipy.sparse as sp
plus1 = sp.coo_matrix(([1, 1, 1], ([0, 0, 1], [0, 1, 0])), shape=(3, 2))
minus1 = sp.coo_matrix(([1], ([2], [1])), shape=(3, 2))
ratings = dgl.heterograph(
    {('user', '+1', 'movie') : plus1,
     ('user', '-1', 'movie') : minus1})

# Creating from networkx graph
import networkx as nx
plus1 = nx.DiGraph()
plus1.add_nodes_from(['u0', 'u1', 'u2'], bipartite=0)
plus1.add_nodes_from(['m0', 'm1'], bipartite=1)
plus1.add_edges_from([('u0', 'm0'), ('u0', 'm1'), ('u1', 'm0')])
# To simplify the example, reuse the minus1 object.
# This also means that you could use different sources of graph data
# for different relationships.
ratings = dgl.heterograph(
    {('user', '+1', 'movie') : plus1,
     ('user', '-1', 'movie') : minus1})

# Creating from edge indices
ratings = dgl.heterograph(
    {('user', '+1', 'movie') : ([0, 0, 1], [0, 1, 0]),
     ('user', '-1', 'movie') : ([2], [1])})

In [3]:
import scipy.io
import urllib.request

data_url = 'https://s3.us-east-2.amazonaws.com/dgl.ai/dataset/ACM.mat'
data_file_path = '/tmp/ACM.mat'

urllib.request.urlretrieve(data_url, data_file_path)
data = scipy.io.loadmat(data_file_path)
print(list(data.keys()))

['__header__', '__version__', '__globals__', 'TvsP', 'PvsA', 'PvsV', 'AvsF', 'VvsC', 'PvsL', 'PvsC', 'A', 'C', 'F', 'L', 'P', 'T', 'V', 'PvsT', 'CNormPvsA', 'RNormPvsA', 'CNormPvsC', 'RNormPvsC', 'CNormPvsT', 'RNormPvsT', 'CNormPvsV', 'RNormPvsV', 'CNormVvsC', 'RNormVvsC', 'CNormAvsF', 'RNormAvsF', 'CNormPvsL', 'RNormPvsL', 'stopwords', 'nPvsT', 'nT', 'CNormnPvsT', 'RNormnPvsT', 'nnPvsT', 'nnT', 'CNormnnPvsT', 'RNormnnPvsT', 'PvsP', 'CNormPvsP', 'RNormPvsP']


In [4]:
print(type(data['PvsA']))
print('#Papers:', data['PvsA'].shape[0])
print('#Authors:', data['PvsA'].shape[1])
print('#Links:', data['PvsA'].nnz)

<class 'scipy.sparse.csc.csc_matrix'>
#Papers: 12499
#Authors: 17431
#Links: 37055


In [5]:
pa_g = dgl.heterograph({('paper', 'written-by', 'author') : data['PvsA']})
# equivalent (shorter) API for creating heterograph with two node types:
pa_g = dgl.bipartite(data['PvsA'], 'paper', 'written-by', 'author')

In [6]:
print('Node types:', pa_g.ntypes)
print('Edge types:', pa_g.etypes)
print('Canonical edge types:', pa_g.canonical_etypes)

# Nodes and edges are assigned integer IDs starting from zero and each type has its own counting.
# To distinguish the nodes and edges of different types, specify the type name as the argument.
print(pa_g.number_of_nodes('paper'))
# Canonical edge type name can be shortened to only one edge type name if it is
# uniquely distinguishable.
print(pa_g.number_of_edges(('paper', 'written-by', 'author')))
print(pa_g.number_of_edges('written-by'))
print(pa_g.successors(1, etype='written-by'))  # get the authors that write paper #1

# Type name argument could be omitted whenever the behavior is unambiguous.
print(pa_g.number_of_edges())  # Only one edge type, the edge type argument could be omitted

Node types: ['paper', 'author']
Edge types: ['written-by']
Canonical edge types: [('paper', 'written-by', 'author')]
12499
37055
37055
tensor([3532, 6421, 8516, 8560])
37055


In [8]:
pa_g.

Graph(num_nodes={'paper': 12499, 'author': 17431},
      num_edges={'written-by': 37055},
      metagraph=[('paper', 'author')])

In [16]:
G = dgl.heterograph({
        ('paper', 'written-by', 'author') : data['PvsA'],
        ('author', 'writing', 'paper') : data['PvsA'].transpose(),
        ('paper', 'citing', 'paper') : data['PvsP'],
        ('paper', 'cited', 'paper') : data['PvsP'].transpose(),
        ('paper', 'is-about', 'subject') : data['PvsL'],
        ('subject', 'has', 'paper') : data['PvsL'].transpose(),
    })

print(G)

Graph(num_nodes={'paper': 12499, 'author': 17431, 'subject': 73},
      num_edges={'written-by': 37055, 'writing': 37055, 'citing': 30789, 'cited': 30789, 'is-about': 12499, 'has': 12499},
      metagraph=[('paper', 'author'), ('paper', 'paper'), ('paper', 'paper'), ('paper', 'subject'), ('author', 'paper'), ('subject', 'paper')])


In [45]:
G.edges()

DGLError: Edge type name must be specified if there are more than one edge types.

In [19]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

pvc = data['PvsC'].tocsr()
# find all papers published in KDD, ICML, VLDB
c_selected = [0, 11, 13]  # KDD, ICML, VLDB
p_selected = pvc[:, c_selected].tocoo()
# generate labels
labels = pvc.indices
labels[labels == 11] = 1
labels[labels == 13] = 2
labels = torch.tensor(labels).long()

# generate train/val/test split
pid = p_selected.row
shuffle = np.random.permutation(pid)
train_idx = torch.tensor(shuffle[0:800]).long()
val_idx = torch.tensor(shuffle[800:900]).long()
test_idx = torch.tensor(shuffle[900:]).long()

In [33]:
pid.shape

(2094,)

In [57]:
G.ndata('paper')

DGLError: Node type name must be specified if there are more than one node types.

In [53]:
G.nodes['paper']

NodeSpace(data={'Wh_written-by': tensor([[0.2342, 0.3199, 0.0658],
        [0.1433, 0.2100, 0.0364],
        [0.1550, 0.2062, 0.0729],
        ...,
        [0.1043, 0.2021, 0.0281],
        [0.1094, 0.2057, 0.0293],
        [0.1095, 0.2057, 0.0293]], grad_fn=<AddmmBackward>), 'Wh_citing': tensor([[ 1.0290, -0.8217, -1.8342],
        [ 0.3535, -1.1083, -0.5215],
        [ 0.6790, -1.7879, -0.2435],
        ...,
        [-0.0411, -0.5388, -0.5951],
        [ 0.0067, -0.5627, -0.6351],
        [ 0.0069, -0.5628, -0.6353]], grad_fn=<AddmmBackward>), 'Wh_cited': tensor([[ 1.2770, -2.3688, -1.2300],
        [ 0.5546, -3.2293,  0.3917],
        [ 0.6866, -4.4502,  1.1324],
        ...,
        [ 0.1296, -2.6325,  0.3993],
        [ 0.1815, -2.6405,  0.3452],
        [ 0.1817, -2.6405,  0.3449]], grad_fn=<AddmmBackward>), 'Wh_is-about': tensor([[ 0.4217,  0.2108, -0.2672],
        [ 0.3612,  0.2423, -0.2083],
        [ 0.3473,  0.3018, -0.1691],
        ...,
        [ 0.3387,  0.1728, -0.2016]

In [48]:
G.edges['citing']

EdgeSpace(data={})

In [43]:
p_selected

[('paper', 'written-by', 'author'),
 ('author', 'writing', 'paper'),
 ('paper', 'citing', 'paper'),
 ('paper', 'cited', 'paper'),
 ('paper', 'is-about', 'subject'),
 ('subject', 'has', 'paper')]

In [11]:
import dgl.function as fn

class HeteroRGCNLayer(nn.Module):
    def __init__(self, in_size, out_size, etypes):
        super(HeteroRGCNLayer, self).__init__()
        # W_r for each relation
        self.weight = nn.ModuleDict({
                name : nn.Linear(in_size, out_size) for name in etypes
            })

    def forward(self, G, feat_dict):
        # The input is a dictionary of node features for each type
        funcs = {}
        for srctype, etype, dsttype in G.canonical_etypes:
            # Compute W_r * h
            Wh = self.weight[etype](feat_dict[srctype])
            # Save it in graph for message passing
            G.nodes[srctype].data['Wh_%s' % etype] = Wh
            # Specify per-relation message passing functions: (message_func, reduce_func).
            # Note that the results are saved to the same destination feature 'h', which
            # hints the type wise reducer for aggregation.
            funcs[etype] = (fn.copy_u('Wh_%s' % etype, 'm'), fn.mean('m', 'h'))
        # Trigger message passing of multiple types.
        # The first argument is the message passing functions for each relation.
        # The second one is the type wise reducer, could be "sum", "max",
        # "min", "mean", "stack"
        G.multi_update_all(funcs, 'sum')
        # return the updated node feature dictionary
        return {ntype : G.nodes[ntype].data['h'] for ntype in G.ntypes}

In [12]:
class HeteroRGCN(nn.Module):
    def __init__(self, G, in_size, hidden_size, out_size):
        super(HeteroRGCN, self).__init__()
        # Use trainable node embeddings as featureless inputs.
        embed_dict = {ntype : nn.Parameter(torch.Tensor(G.number_of_nodes(ntype), in_size))
                      for ntype in G.ntypes}
        for key, embed in embed_dict.items():
            nn.init.xavier_uniform_(embed)
        self.embed = nn.ParameterDict(embed_dict)
        # create layers
        self.layer1 = HeteroRGCNLayer(in_size, hidden_size, G.etypes)
        self.layer2 = HeteroRGCNLayer(hidden_size, out_size, G.etypes)

    def forward(self, G):
        h_dict = self.layer1(G, self.embed)
        h_dict = {k : F.leaky_relu(h) for k, h in h_dict.items()}
        h_dict = self.layer2(G, h_dict)
        # get paper logits
        return h_dict['paper']

In [34]:
# Create the model. The output has three logits for three classes.
model = HeteroRGCN(G, 30, 100, 3)

opt = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

best_val_acc = 0
best_test_acc = 0

for epoch in range(100):
    logits = model(G)
    # The loss is computed only for labeled nodes.
    loss = F.cross_entropy(logits[train_idx], labels[train_idx])

    pred = logits.argmax(1)
    train_acc = (pred[train_idx] == labels[train_idx]).float().mean()
    val_acc = (pred[val_idx] == labels[val_idx]).float().mean()
    test_acc = (pred[test_idx] == labels[test_idx]).float().mean()

    if best_val_acc < val_acc:
        best_val_acc = val_acc
        best_test_acc = test_acc

    opt.zero_grad()
    loss.backward()
    opt.step()

    if epoch % 5 == 0:
        print('Loss %.4f, Train Acc %.4f, Val Acc %.4f (Best %.4f), Test Acc %.4f (Best %.4f)' % (
            loss.item(),
            train_acc.item(),
            val_acc.item(),
            best_val_acc.item(),
            test_acc.item(),
            best_test_acc.item(),
        ))

Loss 1.1330, Train Acc 0.1850, Val Acc 0.2200 (Best 0.2200), Test Acc 0.1876 (Best 0.1876)
Loss 0.7926, Train Acc 0.5362, Val Acc 0.4700 (Best 0.4700), Test Acc 0.4916 (Best 0.4916)
Loss 0.3565, Train Acc 0.9438, Val Acc 0.8000 (Best 0.8000), Test Acc 0.7705 (Best 0.7705)
Loss 0.1062, Train Acc 0.9862, Val Acc 0.8700 (Best 0.8700), Test Acc 0.7848 (Best 0.7881)
Loss 0.0284, Train Acc 1.0000, Val Acc 0.8500 (Best 0.8700), Test Acc 0.7479 (Best 0.7881)
Loss 0.0144, Train Acc 1.0000, Val Acc 0.8400 (Best 0.8700), Test Acc 0.7454 (Best 0.7881)
Loss 0.0125, Train Acc 1.0000, Val Acc 0.8500 (Best 0.8700), Test Acc 0.7596 (Best 0.7881)
Loss 0.0127, Train Acc 1.0000, Val Acc 0.8400 (Best 0.8700), Test Acc 0.7697 (Best 0.7881)
Loss 0.0115, Train Acc 1.0000, Val Acc 0.8500 (Best 0.8700), Test Acc 0.7655 (Best 0.7881)
Loss 0.0108, Train Acc 1.0000, Val Acc 0.8400 (Best 0.8700), Test Acc 0.7680 (Best 0.7881)
Loss 0.0102, Train Acc 1.0000, Val Acc 0.8300 (Best 0.8700), Test Acc 0.7672 (Best 0.7881)